In [134]:
file = r'C:\Users\Trenkin.Sergey\Desktop\wb\WB - Ювелирные изделия - Товары 15012024 - 13022024 14022024.csv'

In [135]:
import pandas as pd
import numpy as np
import requests 
from tqdm import tqdm
from datetime import date
from fake_useragent import UserAgent
import warnings
import time 

warnings.filterwarnings('ignore')

In [136]:
mpstats = pd.read_csv(file, sep=';', on_bad_lines='skip')

In [137]:
old = mpstats.Category.tolist()
category = list()
for cat in old:
    cat = cat.split('/')[-1]
    category.append(cat)

In [138]:
bask = mpstats.Thumb.tolist()
basket = list()
for b in bask:
    b = b.split('/')[:-3]
    if '' in b:
        b.remove('')
    b = f'{b[0]}//{b[1]}/{b[2]}/{b[3]}/{b[4]}/info/ru/card.json'
    basket.append(b)

In [139]:
df = pd.DataFrame({'name': mpstats.Name.tolist(), 
                   'category': category,
                   'url': mpstats.URL.tolist(),
                   'basket': basket,
                   'count': mpstats.Balance.tolist(),
                   'price': mpstats['Final price'].tolist()})

In [140]:
links = df.basket.tolist()

In [141]:
weights = list()
probas = list()
inserts = list()
metals = list()
locks = list()
coatings = list()

In [142]:
for link in tqdm(links):
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    
    try:
        json = requests.get(link, headers=headers, timeout=30, verify=False).json()
    except Exception as ex:
        time.sleep(60)
        json = requests.get(link, headers=headers, timeout=30, verify=False).json()
        
    try:
        for option in json['grouped_options'][0]['options']:
            char = option['name']
            value = option['value']
            if char == 'Состав ювелирного изделия':
                metal = value
            elif char == 'Проба':
                proba = value
            elif char == 'Вставка':
                insert = value
            elif char == 'Минимальный вес (г)':
                weight = value.replace(' г', '')
            elif char == 'Вид замка':
                lock = value
            elif char == 'Покрытие':
                coating = value
    except:
        pass
    
    try:
        for option in json['options']:
            char = option['name']
            value = option['value']
            if char == 'Состав ювелирного изделия':
                metal = value
            elif char == 'Проба':
                proba = value
            elif char == 'Вставка':
                insert = value
            elif char == 'Минимальный вес (г)':
                weight = value.replace(' г', '')
            elif char == 'Вид замка':
                lock = value
            elif char == 'Покрытие':
                coating = value
    except:
        pass 
    
    try:
        brand = json['selling']['brand_name']
    except:
        brand = np.nan
    try:
        weights.append(weight)
    except:
        weights.append(np.nan)
    try:
        probas.append(proba)
    except:
        probas.append(np.nan)
    try:
        inserts.append(insert)
    except:
        inserts.append(np.nan)
    try:
        metals.append(metal)
    except:
        metals.append(np.nan)
    try:
        locks.append(lock)
    except:
        locks.append(np.nan)
    try:
        coatings.append(coating)
    except:
        coatings.append(np.nan)

        
result = pd.DataFrame({'weight': weights,
                       'proba': probas,
                       'insert': inserts,
                       'metal': metals,
                       'lock': locks,
                       'coating': coatings})

  0%|          | 383/117862 [01:07<3:15:03, 10.04it/s] 

In [ ]:
data = pd.concat([df, result], axis=1)

In [ ]:
data.to_excel('wb.xlsx', index=False)